# Web Scraping using Spider

This notebook shows how to use the fastest open 
source web crawler together with AutoGen agents.

First we need to install the Spider SDK and the AutoGen library.

In [1]:
! pip install -qqq pyautogen spider-client

Setting up the LLM configuration and the Spider API key is also required.

In [2]:
import os

config_list = [
    {"model": "gpt-4o", "api_key": os.getenv("OPENAI_API_KEY")},
]

spider_api_key = os.getenv("SPIDER_API_KEY")

Let's define the tool for scraping and crawling data from any website with Spider.
Read more about tool use in this [tutorial chapter](/docs/tutorial/tool-use).

In [3]:
from typing import Any, Dict, List

from spider import Spider
from typing_extensions import Annotated


def scrape_page(
    url: Annotated[str, "The URL of the web page to scrape"],
    params: Annotated[dict, "Dictionary of additional params."] = None,
) -> Annotated[Dict[str, Any], "Scraped content"]:
    # Initialize the Spider client with your API key, if no api key is specified it looks for SPIDER_API_KEY in your environment variables
    client = Spider(spider_api_key)

    if params is None:
        params = {"return_format": "markdown"}

    scraped_data = client.scrape_url(url, params)
    return scraped_data[0]


def crawl_page(
    url: Annotated[str, "The url of the domain to be crawled"],
    params: Annotated[dict, "Dictionary of additional params."] = None,
) -> Annotated[List[Dict[str, Any]], "Scraped content"]:
    # Initialize the Spider client with your API key, if no api key is specified it looks for SPIDER_API_KEY in your environment variables
    client = Spider(spider_api_key)

    if params is None:
        params = {"return_format": "markdown"}

    crawled_data = client.crawl_url(url, params)
    return crawled_data

[{'content': 'Spider - The Fastest Web Crawling Service[Spider v1 Logo Spider ](/)[Pricing](/credits/new)[GitHub](https://github.com/spider-rs/spider) [Twitter](https://twitter.com/spider_rust) Toggle ThemeSign InRegisterTo help you get started with Spider, we’ll give you $200 in credits when you spend $100. [Get Credits](/credits/new)LangChain integration [now available](https://python.langchain.com/docs/integrations/document_loaders/spider)The World\'s Fastest and Cheapest Crawler API==========View Demo* Basic* StreamingExample requestPythonCopy```import requests, osheaders = {    \'Authorization\': os.environ["SPIDER_API_KEY"],    \'Content-Type\': \'application/json\',}json_data = {"limit":50,"url":"http://www.example.com"}response = requests.post(\'https://api.spider.cloud/crawl\',  headers=headers,  json=json_data)print(response.json())```Example ResponseUnmatched Speed----------### 2.5secs  ###To crawl 200 pages### 100-500x  ###Faster than alternatives### 500x  ###Cheaper than t

Create the agents and register the tool.

In [4]:
from autogen import ConversableAgent, register_function

# Create web scraper agent.
scraper_agent = ConversableAgent(
    "WebScraper",
    llm_config={"config_list": config_list},
    system_message="You are a web scraper and you can scrape any web page to retrieve its contents."
    "Returns 'TERMINATE' when the scraping is done.",
)

# Create web crawler agent.
crawler_agent = ConversableAgent(
    "WebCrawler",
    llm_config={"config_list": config_list},
    system_message="You are a web crawler and you can crawl any page with deeper crawling following subpages."
    "Returns 'TERMINATE' when the scraping is done.",
)

# Create user proxy agent.
user_proxy_agent = ConversableAgent(
    "UserProxy",
    llm_config=False,  # No LLM for this agent.
    human_input_mode="NEVER",
    code_execution_config=False,  # No code execution for this agent.
    is_termination_msg=lambda x: x.get("content", "") is not None and "terminate" in x["content"].lower(),
    default_auto_reply="Please continue if not finished, otherwise return 'TERMINATE'.",
)

# Register the functions with the agents.
register_function(
    scrape_page,
    caller=scraper_agent,
    executor=user_proxy_agent,
    name="scrape_page",
    description="Scrape a web page and return the content.",
)

register_function(
    crawl_page,
    caller=crawler_agent,
    executor=user_proxy_agent,
    name="crawl_page",
    description="Crawl an entire domain, following subpages and return the content.",
)

Start the conversation for scraping web data. We used the
`reflection_with_llm` option for summary method
to perform the formatting of the output into a desired format.
The summary method is called after the conversation is completed
given the complete history of the conversation.

In [5]:
# Scrape page
scraped_chat_result = user_proxy_agent.initiate_chat(
    scraper_agent,
    message="Can you scrape william-espegren.com for me?",
    summary_method="reflection_with_llm",
    summary_args={"summary_prompt": """Summarize the scraped content"""},
)

UserProxy (to WebScraper):

Can you scrape william-espegren.com for me?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
WebScraper (to UserProxy):

***** Suggested tool call (call_qCNYeQCfIPZkUCKejQmm5EhC): scrape_page *****
Arguments: 
{"url":"https://www.william-espegren.com"}
****************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION scrape_page...
UserProxy (to WebScraper):

UserProxy (to WebScraper):

***** Response from calling tool (call_qCNYeQCfIPZkUCKejQmm5EhC) *****
[{"content": "William Espegren - Portfoliokeep scrollingMADE WITHCSS, JSMADE BYUppsalaWilliam EspegrenWith \u00b7LoveOpen For Projects[CONTACT ME](https://www.linkedin.com/in/william-espegren/)[Instagram](https://www.instagram.com/williamespegren/)[LinkedIn](https://www.linkedin.com/in/william-espegren/)[Twitter](https://twi

In [6]:
# Crawl page
crawled_chat_result = user_proxy_agent.initiate_chat(
    crawler_agent,
    message="Can you crawl william-espegren.com for me, I want the whole domains information?",
    summary_method="reflection_with_llm",
    summary_args={"summary_prompt": """Summarize the crawled content"""},
)

UserProxy (to WebCrawler):

Can you crawl william-espegren.com for me, I want the whole domains information?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
WebCrawler (to UserProxy):

***** Suggested tool call (call_0FkTtsxBtA0SbChm1PX085Vk): crawl_page *****
Arguments: 
{"url":"http://www.william-espegren.com"}
***************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION crawl_page...
UserProxy (to WebCrawler):

UserProxy (to WebCrawler):

***** Response from calling tool (call_0FkTtsxBtA0SbChm1PX085Vk) *****
[{"content": "William Espegren - Portfoliokeep scrollingMADE WITHCSS, JSMADE BYUppsalaWilliam EspegrenWith \u00b7LoveOpen For Projects[CONTACT ME](https://www.linkedin.com/in/william-espegren/)[Instagram](https://www.instagram.com/williamespegren/)[LinkedIn](https://www.linkedin.com/in/willia

The output is stored in the summary.

In [7]:
print(scraped_chat_result.summary)
# print(crawled_chat_result.summary) # We show one for cleaner output

The website belongs to William Espegren, who is based in Uppsala and possesses skills in CSS and JavaScript. He is open to new projects. You can contact him through the following links:

- [LinkedIn](https://www.linkedin.com/in/william-espegren/)
- [Instagram](https://www.instagram.com/williamespegren/)
- [Twitter](https://twitter.com/WilliamEspegren)
- [GitHub](https://github.com/WilliamEspegren)

Feel free to reach out to him for project collaborations.
